In [1]:
#outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/single/"
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/"
#cloneID = "0_C6"


In [2]:
from os.path import join, exists, dirname
from glob import glob
import pickle
import mplh.cluster_help as ch
import mplh.fig_utils as fu

import os
import vireoSNP
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
from scipy.stats import hypergeom
print(vireoSNP.__version__)

import pandas as pd
import seaborn as sns
from vireoSNP import Vireo
np.set_printoptions(formatter={'float': lambda x: format(x, '.5f')})

from svgutils.compose import *


import svgutils.transform as sg
import sys 


import src.utils.fig_concat as fc

from os.path import basename

fig_utils from mplh
0.5.6


In [3]:
out_svg_f = join(outdir,'clone_shift_combine.svg')
out_pdf_f = join(outdir,'clone_shift_combine.pdf')

out_vars_svg_f = join(outdir,'clone_variants_combine.svg')
out_vars_pdf_f = join(outdir,'clone_variants_combine.pdf')

f1_umap_clust = join(outdir,'top_umap_clust_col.svg')
f2_umap_cond = join(outdir,'top_umap_cond_col.svg')
f3_clone_clust = join(outdir,'top_cluster_condition_ncells.svg')
f4_hyper = join(outdir,'top10_sig_hypergeo.svg')

fvars_pct = join(outdir, "top_variants_distinct_pct_vs_other.svg")
fvars_pct_minus = join(outdir, "top_variants_pct_minus_other.svg")
fvars_vars_af = join(outdir, "top_variants_af.svg")
fvars_vars_af_box = join(outdir, "top_variants_in_clone_af_boxen.svg") 


In [8]:
#tmp_out_svg_f = join(outdir,'tmp_clone_shift_combine.svg')
figure_names = {"umap_clust":f1_umap_clust, "umap_cond":f2_umap_cond, 
                "clone_lineage": f3_clone_clust, "hyper": f4_hyper}
#{figure_names[key]:key for key in figure_names}
svg_df = pd.DataFrame(pd.Series(figure_names), 
                      columns=["Figure"]).transpose()
svg_df


# svgs = fc.files_to_svg_dict(svg_df, scale=0.25, verbose=False)
# svgs
fc.main_df(svg_df, out_f=out_svg_f, scale=0.25, figure_names=figure_names, verbose=True)


/data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/top_umap_clust_col.svg
1440.00pt
height
5760.00pt
/data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/top_umap_cond_col.svg
1440.00pt
height
5760.00pt
/data/Mito_Trace/output/pipeline/v04/CHIP_b2/InputOnly/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/top_cluster_condition_ncells.svg
626.375pt
height
2929.664062pt
/data/

In [11]:
svg_df

,umap_clust,umap_cond,clone_lineage,hyper
Figure,top_umap_clust_col.svg,top_umap_cond_col.svg,top_cluster_condition_ncells.svg,top10_sig_hypergeo.svg


In [5]:

# svg_df

# curr_init_files = []
# curr_ref = svg_df.iloc[0,0].split('.svg')[0]
# curr_keys = [x.split('.svg')[0] for x in svg_df.iloc[0].values]
# fc.rescale_height(svgs, curr_keys, curr_ref)


# def change_positions(df, svgs, widths, heights):
#     """Move the images to the desired positions."""
#     for i_row, row in enumerate(df.index):
#         for i_col, col in enumerate(df.columns):
#             curr = df.loc[row, col].split(".svg")[0]
#             svgs[curr].move(sum(widths[:i_col]), sum(heights[:i_row]))

# col_max_widths = []
# row_max_heights = [0]*len(svg_df.columns)  # should be the same in theory
# all_heights = []
# for col in svg_df.columns:
#     col_max_widths.append(svgs[svg_df.iloc[0][col].split(".svg")[0]].width)
#     all_heights.append(svgs[svg_df.iloc[0][col].split(".svg")[0]].height)
    
# fc.change_positions(svg_df, svgs, col_max_widths, row_max_heights)

# full_width = sum(col_max_widths)
# full_height = sum(all_heights)
# fig = sg.SVGFigure(full_width + 15, full_height + 15)

# fig.append([s.data for s in svgs.values()])
# text = fc.letter_annotations(svgs, df=svg_df, figure_names=figure_names,
#                           transposed=True)
# fig.append(text)
# fig.save(out_svg_f)

# cmd = f'inkscape --file={out_svg_f} -d 300 --export-area-drawing --without-gui --export-png={out_svg_f.replace(".svg", ".png")}'  # --export-area-drawing --without-gui --export-psvg_df=output.psvg_df'
# print(cmd)
# os.system(cmd)



# col_max_widths